# Formatter

In [1]:
import os
import shutil
import random

In [2]:
root_dir = '../../Textures'
output_dir = os.path.join(root_dir, 'Dataset_Textures_Train_Undersampled')

In [3]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15
random.seed(42)

In [4]:
os.makedirs(output_dir, exist_ok=True)

In [32]:
folders_to_unify = [
    # '2D_Municipio', 
    # '2D_Palazzo_Maresa', 
    # '2D_Rilievi_Centro_Italia', 
    'Textures_Tiled_4096_SantElia_Rudere_CentroItalia', 
    'Textures_Tiled_4096_Torre_Fontecchio_CentroItalia', 
    'Textures_Tiled_4096_Corvara Torre_CentroItalia', 
    'Textures_Tiled_4096_Municipio_Tredozio', 
    'Textures_Tiled_4096_Palazzo_Fantini_Tredozio', 
    'Textures_Tiled_4096_Palazzo_Maresa_Tredozio', 
    'Textures_Tiled_4096_Piazza_Tredozio', 
    'Textures_Tiled_4096_PiccoloRudere_Corvara_CentroItalia', 
    'Textures_Tiled_4096_Poggio di Roio-Ed3_CentroItalia', 
    'Textures_Tiled_4096_Ripa_Chiesetta_CentroItalia', 
    'Textures_Tiled_4096_Scuola_Tredozio', 
    'Textures_Tiled_4096_ChiesaSconsacrata_Navelli_CentroItalia', 
    'Textures_Tiled_4096_SantaMariadelMontePalazzina_CentroItalia', 
    'Textures_Tiled_4096_SantaMariadelPonte_Chiesa_CentroItalia'
]

In [ ]:
# folders_to_unify = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
print(folders_to_unify)

['Dataset_Textures_Train_Undersampled', 'Testures_Tiled_4096_SantElia_Rudere_CentroItalia', 'Testures_Tiled_4096_Torre_Fontecchio_CentroItalia', 'Textures_Tiled_4096_Corvara Torre_CentroItalia', 'Textures_Tiled_4096_Municipio_Tredozio', 'Textures_Tiled_4096_Palazzo_Fantini_Tredozio', 'Textures_Tiled_4096_PiccoloRudere_Corvara_CentroItalia', 'Textures_Tiled_4096_Poggio di Roio-Ed3_CentroItalia', 'Textures_Tiled_4096_Ripa_Chiesetta_CentroItalia', 'Textures_Tiled_4096_Scuola_Tredozio', 'Texture_Tiled_4096_ChiesaSconsacrata_Navelli_CentroItalia', 'Texture_Tiled_4096_SanNicandroCasolare_CentroItalia', 'Texture_Tiled_4096_SantaMariadelMontePalazzina_CentroItalia', 'Texture_Tiled_4096_SantaMariadelPonte_Chiesa_CentroItalia']


In [ ]:
for folder in folders_to_unify:
    folder_path = os.path.join(root_dir, folder, 'undersampled')
    if not os.path.exists(folder_path):
        print(f"Folder {folder} does not exist in {root_dir}. Skipping.")
        continue

    output_folder_path = os.path.join(output_dir, folder)
    for split in ['train', 'val', 'test']:
        for sub in ['imgs', 'labels']:
            os.makedirs(os.path.join(output_folder_path, split, sub), exist_ok=True)

    imgs_path = os.path.join(folder_path, 'imgs')
    labels_path = os.path.join(folder_path, 'labels')

    if not (os.path.isdir(imgs_path) and os.path.isdir(labels_path)):
        print(f"Missing imgs or labels in {folder_path}. Skipping.")
        continue

    image_files = [f for f in os.listdir(imgs_path) if f.endswith('.jpg')]
    base_names = [os.path.splitext(f)[0] for f in image_files]
    base_names = [b for b in base_names if os.path.exists(os.path.join(labels_path, b + '.png'))]

    random.shuffle(base_names)

    total = len(base_names)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    split_names = {
        'train': base_names[:train_end],
        'val': base_names[train_end:val_end],
        'test': base_names[val_end:]
    }

    for split, names in split_names.items():
        for name in names:
            img_src = os.path.join(imgs_path, name + '.jpg')
            label_src = os.path.join(labels_path, name + '.png')

            img_dst = os.path.join(output_folder_path, split, 'imgs', name + '.jpg')
            label_dst = os.path.join(output_folder_path, split, 'labels', name + '.png')

            shutil.copy2(img_src, img_dst)
            shutil.copy2(label_src, label_dst)